<a href="https://www.kaggle.com/code/lazuardialmuzaki/eda-and-time-series-forecasting-retail-store?scriptVersionId=142689205" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Exploratory Data Analysis (with Choropleth Map) and Sales Time Series Forecasting of SuperStore Retail 

## 1. Load Data

The Data used is about Superstore Sales Dataset during 2015 through 2018. Superstore is one of the most popular retail store based in United States of America.

In [ ]:
import pandas as pd

raw_data = pd.read_csv('/kaggle/input/superstore-sales-dataset/store.csv')

In [ ]:
raw_data.head(5)

## 2. Data Pre-Processing

### 2.1 Checking Missing Values
Checking missing values are needed so the data used will not be disrupted by row that has null values on column we are interested to analyze

In [ ]:
raw_data.isnull().sum()

Since the null values only exist in Postal Code which is a column we are not important to our analysis, it will be fine to let the missing values as it be. 

In [ ]:
# Dropping the column 'Row ID', as it does not help much in the process of data analysis of the dataset.
raw_data.drop('Row ID',axis = 1, inplace = True)

## 3. Extracting Date Information

Date information are essential as our analysis mainly will discover insight regarding time and location.

**Converting Date column type**


In [ ]:
raw_data['Order Date'] = pd.to_datetime(raw_data['Order Date'], format='%d/%m/%Y')  #converting the data type of 'Order Date' column to date time format
raw_data['Ship Date'] = pd.to_datetime(raw_data['Ship Date'], format='%d/%m/%Y')  #converting the data type of 'Ship Date' column to date time format
raw_data.info()

**Assign new columns to store Year, Month, and Day information**

In [ ]:
import numpy as np

raw_data['Order Year'] = pd.to_datetime(raw_data['Order Date']).dt.year #store year information

raw_data['Order Month'] = pd.to_datetime(raw_data['Order Date']).dt.month #store month numerical index information
raw_data['Order MonthName'] = pd.to_datetime(raw_data['Order Date']).dt.month_name() # store month name information

raw_data['Order Day'] = pd.to_datetime(raw_data['Order Date']).dt.day # store day numerical index information
raw_data['Order DayName'] = pd.to_datetime(raw_data['Order Date']).dt.day_name() # store day name information

In [ ]:
raw_data.head()

Check the DataFrame, it can be seen now several new columns have been added

## 4. Exploratory Data Analysis

Explorator Data Analysis (EDA) is used to analyze, investigate and summarize main characteristics of datasets through visualization methods. On this ocassion, EDA is specifically used to see if the sales by Superstore may or may not follow some **seasonal trends**.

Based on the columns/features that are given from the DataFrame, it is decided that some EDA specifically designed to see the relationship between accumulation of sales with certain variable which are:

- Sales over Time (Year, Month, Day)
- Sales over Category (Goods category, Region, State, City)

### 4.1 EDA: Using Line Chart to Examine Sales over Time

Sales over time examined through the accumulation of sales over 4 years (2015-2018) vs year, month and day.

In [ ]:
# Self-defined function is being introduced in order to ease the process of creating line chart

import seaborn as sns
import matplotlib.pyplot as plt

def linesales(df, category, sort_by, title=None):
    sns.lineplot(data=df.groupby([category]).sum("Sales").sort_values(sort_by, ascending=True)['Sales'])
    plt.xticks(
    rotation=45, 
    horizontalalignment='right',
    fontweight='light',
    fontsize='x-large')
    plt.title(label=title, fontdict={'fontsize': 15})

**Plotting Sales over Years, Months, and Days**

In [ ]:
# Subplots summoned as foundation for later charts

plt.subplots(3,1, figsize=(8,14))


plt.subplot(3,1,1)
linesales(raw_data, 'Order Year', 'Order Year', 'Total Sales over Years')
plt.tight_layout()

plt.subplot(3,1,2)
linesales(raw_data, 'Order MonthName', 'Order Month', "Average Sales per Month")
plt.tight_layout()

plt.subplot(3,1,3)
linesales(raw_data, 'Order DayName', 'Order Day', "Average Sales per Day")
plt.tight_layout()



From graph above, it could be seen that:

- First figure of 'Total Sales over Years' depicts  how the last two years have recorded significant steady rise. From around 480,000 sales in 2015, it was dropped slightly to approximately 460,000 a year later. During 2017 and 2018, it shows a promising rise to around 600,000 and 720,000 sales respectively

- Second figure of 'Total Sales per Month' strongly indicates that there is a trend during last four months of the year (September to December) in which the sales rising convincingly

- Third figure of 'Total Sales per Day' shows that two days of weekends have higher sales (Saturday & Sunday) compared to all days of weekdays except Tuesday which may be indicating another kind of trend.


### 4.2 EDA: Using Bar Chart and Map to Describe Sales over Category

Sales over category is examined through the accumulation of sales over all years combined vs category of goods, region, city and states.

Before we begin, total amounts of category level is examined for every category that will be analyzed.

In [ ]:
print("Category of Goods: "+str(len(raw_data['Category'].unique())))
print("Region: "+str(len(raw_data['Region'].unique())))
print("State: "+str(len(raw_data['State'].unique())))
print("City: "+str(len(raw_data['City'].unique())))


With this result, it will be convenient to explain using bar chart for 'Category of Goods' and 'Region' as they have small level of category.

For 'State', it might be best to take benefit of geographic map of United States to show the sales accumulation comparison.

However for 'City', there are so many class. For that, bar chart is the one which is used but with modification that only top 10 selling city will be included in the chart.

#### Plotting Sales over Category of Goods and Region using Bar Chart

In [ ]:
# self-defined function is developed to ease the creation of bar chart

def barsales(df, category, title=None):
    data = df.groupby([category])['Sales'].sum().reset_index().sort_values('Sales', ascending=True)
    sns.barplot(data= data, y=data[category], x=data['Sales'])
    plt.xticks(
    rotation=45, 
    horizontalalignment='right',
    fontweight='light',
    fontsize='x-large')
    plt.title(label=title)

In [ ]:
plt.subplots(2,1, figsize=(8,10))


plt.subplot(2,1,1)
barsales(raw_data, 'Category', 'Overall Sales over Category of Goods')
plt.tight_layout()

plt.subplot(2,1,2)
barsales(raw_data, 'Region', 'Overall Sales over Region') 
plt.tight_layout()

From result above, it is apparent that for 'Category of Goods', 'Technology' is one best-selling type of product. but not necessarily significant compared to the other two type of product. 

For 'Region', West region has gotten the most sales followed by East, Central, and South respectively.

#### Plotting Sales over States

As mentioned earlier, different approach is being used to describe Sales over State which is by using geographical map.
In doing so, additional column information is being integrated and designated dataframe is being prepared 

In [ ]:
# Creating list for name of State and State Code which later used as new column of designated dataframe

state = ['Alabama', 'Arizona' ,'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 
         'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
         'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana','Nebraska', 'Nevada', 'New Hampshire',
         'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
         'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
         'West Virginia', 'Wisconsin','Wyoming']
state_code = ['AL','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA',
              'MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN',
              'TX','UT','VT','VA','WA','WV','WI','WY']

In [ ]:
# Creating a designated dataframe for later map plotting

state_df = pd.DataFrame(state, state_code) # Create a dataframe
state_df.reset_index(level=0, inplace=True)
state_df.columns = ['State Code','State']
sales = raw_data.groupby(["State"]).sum("Sales").sort_values("Sales", ascending=False)
sales.reset_index(level=0, inplace=True)
sales.drop(labels=['Postal Code','Order Year','Order Month','Order Day'],axis=1, inplace = True)
sales= sales.sort_values('State', ascending=True)
sales.reset_index(inplace = True)
sales.drop(labels='index',axis=1,inplace = True)
sales.insert(1, 'State Code', state_df['State Code'])

In [ ]:
sales

After the DataFrame is ready, now we start to develop the function to plot a map based on this DataFrame.

The map used here is **Choropleth Map**, a type of statistical thematic map that uses pseudocolor corresponding with an aggregate summary of a geographic characteristic within spatial enumeration units. For this case, it will be about Sales accumulation of a state.

In [ ]:
# Implementing the designated DataFrame to Choropleth Map

import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, plot_mpl

def plotmap(category, title):
    sales['text'] = sales[category]
    fig = go.Figure(data=go.Choropleth(
        locations=sales['State Code'], # Spatial coordinates
        text=sales['text'],
        z = sales['Sales'].astype(float), # Data to be color-coded
        locationmode = 'USA-states', # set of locations match entries in `locations`
        colorscale = 'Greens',
        colorbar_title = "Sales",
    ))
    fig.update_layout(
        title_text = title,
        geo_scope='usa', # limite map scope to USA
    )
    fig.show();
    
plotmap('State','Sales over States')

Choropleth Map above explains how different gradient of green color exemplify different magnitude of total sales. California has the most sales over all years with around 446,000 sales, followed by New York and Texas (around 306,000 and 169,000 respectively)

#### Plotting Sales over City

For City category, Top 10 Cities are selected and then will feature on bar chart.

In [ ]:
city_df=raw_data.groupby("City").sum("Sales").sort_values("Sales", ascending = False)
city_df = city_df[["Sales"]]
city_df.reset_index(inplace=True)


merged_city_state = pd.merge(city_df, raw_data[['City','State']], how="inner", on=["City"])
merged_city_state = merged_city_state.drop_duplicates(subset='City')
merged_city_state['City,State'] = merged_city_state['City'] + ", " + merged_city_state['State']

sns.barplot(x = "Sales" , y = "City,State", data = merged_city_state[:10],edgecolor = "white", palette = "pastel")
plt.xlabel("Sales", size = 20,color='black')

plt.ylabel("City, State", size = 20,color='black')
plt.title("Top_10_Cities in Sales", size = 20, pad = 20,color='black')
plt.show()

From result above, we have Top 10 Cities in Sales which stakeholder might want to focus more on. New York City in New York state is shown dominating with total sales of approximately 250,000 sales. 

## 5. Time Series Analysis
Time series analysis comprises methods for analyzing time series data in order to extract meaningful statistics and other characteristics of the data. In this case, we want to perform time series analysis in order to be able to predict the sales performance to avoid 'out of stock' situation in future

In [ ]:
#import the necessary Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

### 5.1 Dataframe Preparation

In [ ]:
#sorting data by order date
raw_data.sort_values(by=['Order Date'], inplace=True, ascending=True) #Sorting data by  ascending order of the coloumn values 'Order Date'
raw_data.set_index("Order Date", inplace = True) #Setting 'Order Date' as index of the dataframe 'df' for ease of Time Series Analysis

In [ ]:
# To forecast sales seven days later of the order date, let us create a new dataframe with only the target column i.e,
# the 'Sales' column and 'Order Date' as the index

new_data = pd.DataFrame(raw_data['Sales'])
new_data

In [ ]:
#Plotting the data to understand the sales distribution from the year 2015-2018
new_data.plot(figsize=(20, 12))
plt.show()

### 5.2 Stationarity Tests
A series is said to be stationary when its mean and variance do not change over time. From the above distribution of the sales it is not clear whether the sales distribution is stationary or not. Let us perform some stationarity tests to check whether the time series is stationary or not.

In [ ]:
#Checkting for Stationarity
new_data =  pd.DataFrame(new_data['Sales'].resample('D').mean())
new_data = new_data.interpolate(method='linear') #The interpolate() function is used to interpolate values according to
#different methods. It ignore the index and treats the values as equally spaced.

In [ ]:
# To check for stationarity by comparing the change in mean and variance over time, let us split the data into train, test and validate.
train, test, validate = np.split(new_data['Sales'].sample(frac=1), [int(.6*len(new_data['Sales'])),int(.8*len(new_data['Sales']))])

In [ ]:
print('Train Dataset')
print(train)
print('Test Dataset')
print(test)
print('Validate Dataset')
print(validate)

In [ ]:
mean1, mean2, mean3 = train.mean(), test.mean(), validate.mean() #taking mean of train, test and validate data
var1, var2, var3 = train.var(), test.var(), validate.var() #taking variance of train, test and validate data

print('Mean:')
print(mean1, mean2, mean3)
print('Variance:')
print(var1, var2, var3)

From the above values of mean and variance, it can be inferred that their is not much difference in the three values of mean and variance, indicating that the series is stationary. However, to verify our observations, let us perform a standard stationarity test, called Augmented Dicky Fuller test.

### 5.3 Augmented Dicky Fuller test
The Augmented Dickey-Fuller test is a type of statistical test alsocalled a unit root test.The base of unit root test is that it helps in determining how strongly a time series is defined by a trend.

The null hypothesis of the test is that the time series can be represented by a unit root, that it is not stationary. The alternate hypothesis (rejecting the null hypothesis) is that the time series is stationary.

1.Null Hypothesis(H0): Time series is not stationary
2.Alternate Hypothesis (H1): Time series is stationary

This result is interpreted using the p-value from the test.

1.p-value > 0.05: Fail to reject the null hypothesis (H0), the data has a unit root and is non-stationary.
2.p-value <= 0.05: Reject the null hypothesis (H0), the data does not have a unit root and is stationary.

In [ ]:
# Method 2
# Augmented Dicky Fuller Test

from statsmodels.tsa.stattools import adfuller #importing adfuller tool from statsmodels
#statsmodels provide adfuller() fucntion to implement stationarity test of a time series

adf = adfuller(new_data)

print(adf)
print('\nADF = ', str(adf[0])) #more towards negative value the better
print('\np-value = ', str(adf[1]))
print('\nCritical Values: ')

for key, val in adf[4].items(): #for loop to print the p-value (1%, 5% and 10%) and their respective values
    print(key,':',val)


    if adf[0] < val:
        print('Null Hypothesis Rejected. Time Series is Stationary')
    else:
        print('Null Hypothesis Accepted. Time Series is not Stationary')

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 30, 20

import statsmodels.api as sm
decomposition = sm.tsa.seasonal_decompose(new_data, model='additive') #function used to decompose Time Series Data into Trend and Seasonality
fig = decomposition.plot()
plt.show()

Now that we know our time series is data is stationary. Let us begin with model training for forecasting the sales. We have chosen SARIMA model to forecast the sales.

Seasonal Autoregressive Integrated Moving Average, SARIMA or Seasonal ARIMA, is an extension of ARIMA that supports univariate time series data with a seasonal component.

SARIMA requires selecting hyperparameters for both the trend and seasonal elements of the series.

Trend Elements There are three trend elements that require configuration.
p: Trend autoregression order. d: Trend difference order. q: Trend moving average order.

Seasonal Elements There are four seasonal elements:
P: Seasonal autoregressive order. D: Seasonal difference order. Q: Seasonal moving average order. m: The number of time steps for a single seasonal period.

The notation for a SARIMA model is specified as: SARIMA(p,d,q)(P,D,Q)m

### 5.4 SARIMA model

In [ ]:
import itertools
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq_comb = [(i[0], i[1], i[2], 12) for i in list(itertools.product(p, d, q))] #for loop for creating combinations of seasonal parameters of SARIMA
print('Examples of parameter combinations for Seasonal ARIMA:')
print('SARIMA: {} x {}'.format(pdq[1], seasonal_pdq_comb[1]))
print('SARIMA: {} x {}'.format(pdq[1], seasonal_pdq_comb[2]))
print('SARIMA: {} x {}'.format(pdq[2], seasonal_pdq_comb[3]))
print('SARIMA: {} x {}'.format(pdq[2], seasonal_pdq_comb[4]))

In [ ]:
for parameters in pdq: #for loop for determining the best combination of seasonal parameters for SARIMA
    for seasonal_param in seasonal_pdq_comb:
        try:
            mod = sm.tsa.statespace.SARIMAX(new_data,
                                            order=parameters,
                                            seasonal_param_order=seasonal_param,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False) #determines the AIC value of the model**
            results = mod.fit(disp=0)
            print('SARIMA{}x{}12 - AIC:{}'.format(parameters, seasonal_param, results.aic))
        except:
            continue

**The Akaike information criterion (AIC) is an estimator of out-of-sample prediction error and thereby relative**
quality of statistical models for a given set of data. AIC estimates the relative amount of information lost
by a given model. The less information a model loses, the higher the quality of that model.

In [ ]:
# After choosing the combination of seasonal parameters with least AIC value, let us train the SARIMA model
mod = sm.tsa.statespace.SARIMAX(new_data,
                                order=(1, 1, 1),
                                seasonal_order=(1, 1, 1, 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False) #model defintion
results = mod.fit(disp=0) #model fitting
print(results.summary().tables[1]) # displaying the result

In [ ]:
results.plot_diagnostics(figsize=(16, 8))
plt.show()

Produces a plot grid of:
1. Standardized residuals over time
2. Histogram plus estimated density of standardized residulas and along with a Normal(0,1) density plotted for reference.
3. Normal Q-Q plot, with Normal reference line
4. Correlogram.

In [ ]:
pred = results.get_prediction(start=pd.to_datetime('2015-01-03'), dynamic=False) # variable to display plot for predicted values
pred_val = pred.conf_int()
ax = new_data['2014':].plot(label='observed') # displays plot for original values
pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.7, figsize=(40, 26)) # displays plot for predicted values
ax.fill_between(pred_val.index,
                pred_val.iloc[:, 0],
                pred_val.iloc[:, 1], color='k', alpha=.2)
ax.set_xlabel('Date')
ax.set_ylabel('Sales')
plt.legend()
plt.show()

In [ ]:
y_forecasted = pred.predicted_mean
y_truth = new_data['Sales']

from sklearn.metrics import mean_squared_error
from math import sqrt

mse = mean_squared_error(y_forecasted, y_truth)
rmse = sqrt(mse)
print('The Mean Squared Error of the forecasts is {}'.format(round(rmse, 2))) # displays the root mean squared error of the forecast with rounding it up to 2 decimals

### 5.5 Out of Sample Forecast
To forecast sales values after some time period of the given data. In our case, we have to forecast sales with time period of 7 days.

In [ ]:
forecast_data = results.forecast(steps=30) # making a forecast of a month later of the last date in the 'Order Date' column
print(forecast_data.astype('int')) #displays the sales forecast as type integer
def forecast(data):
    data.plot(figsize=(20, 12))
    plt.title('Figure 2. One Month Forecast of Sales Performance')
    plt.show()
forecast(forecast_data)

# Study Case Summary

Superstore is one of popular retail store that operates across United States. But recently, the stakeholders are informed that in overall, some stores have been encountering case of 'out of stock'. The stakeholders then asked the data analytics department to examine the 4-year sales historical data to gain more detailed description of sales performance. 

Exploratory Data Analysis (EDA) then is implemented to show how sales changes over the years, months and days indicating the trends that might have been existed incorporating customer buying behaviour. Sales performance evaluation over different region, states and city are also performed utilizing charts and map to obtain clear picture to distinguish best-selling area that might become a focus for stakeholder to improve. First figure below shows how sales performance differs across the states.

Furthermore, a time series analysis is also conducted based on the SARIMA model to predict the sales performance for upcoming period to avoid ‘out of stock’ situation. Example on second figure below shows an overall sales prediction one month after the last time-node of the dataset. It is found that the sales volume has several maximum values, but as time goes by, it gradually tends to be stable. According to the prediction results of the model, the daily sales volume of the store in overall will be stable at around 230 units.

The sales prediction can be modified to specific criteria depends on needs (for example: monthly prediction for New York City), but the challenge will be about preparing the data prior to the time series analysis.


In [ ]:
plotmap('State', 'Figure 1. Sales over States')

In [ ]:
forecast(forecast_data)
